In [72]:
#Imports
import math
import csv
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg as lag

csv_arr = []

#The csv reader
with open('cp2_input.csv') as csv_in:
    filereader = csv.reader(csv_in)
    for row in filereader:
        csv_arr.append(row)

two_sig_a = [float(i) for i in csv_arr[0]]
#The fission and inscatter matrix rows were precalculated and entered by row
two_fiss_row1 = [float(i) for i in csv_arr[1]]
two_fiss_row2 = [float(i) for i in csv_arr[2]]
two_in_row1 = [float(i) for i in csv_arr[3]]
two_in_row2 = [float(i) for i in csv_arr[4]]
#The outscatter values were also precalculated
two_out = [float(i) for i in csv_arr[5]]
eight_sig_a = [float(i) for i in csv_arr[6]]
#There is most definitly a better way to do this, but this makes the fission and inscatter matrices easily defined down the line
eight_fiss_row1 = [float(i) for i in csv_arr[7]]
eight_fiss_row2 = [float(i) for i in csv_arr[8]]
eight_fiss_row3 = [float(i) for i in csv_arr[9]]
eight_in_row1 = [float(i) for i in csv_arr[10]]
eight_in_row2 = [float(i) for i in csv_arr[11]]
eight_in_row3 = [float(i) for i in csv_arr[12]]
eight_in_row4 = [float(i) for i in csv_arr[13]]
eight_in_row5 = [float(i) for i in csv_arr[14]]
eight_in_row6 = [float(i) for i in csv_arr[15]]
eight_in_row7 = [float(i) for i in csv_arr[16]]
eight_in_row8 = [float(i) for i in csv_arr[17]]
eight_out = [float(i) for i in csv_arr[18]]
#Xi values at group 4 onward were 0, so those values are zero too
zero = list(np.zeros(8))

#This takes outscattering values and makes an ordered array of them
def outscattering(array):
    #There is definitely a way to do this with the identity matrix, but this'll work for a quick fix
    outscatter = np.zeros((len(array),len(array)))
    for i in range(len(array)):
        outscatter[i][i] = array[i]
    return outscatter

#Defining the matrices
two_outscatter = outscattering(two_out)
two_absorption = outscattering(two_sig_a)
two_inscatter = [two_in_row1,two_in_row2]
two_fission = [two_fiss_row1,two_fiss_row2]
eight_absorption = outscattering(eight_sig_a)
#These two lines are why I did it by row, making matrices with complicated structures very easily defined
eight_fission = [eight_fiss_row1,eight_fiss_row2,eight_fiss_row3,zero,zero,zero,zero,zero]
eight_inscatter = [eight_in_row1,eight_in_row2,eight_in_row3,eight_in_row4,eight_in_row5,eight_in_row6,eight_in_row7,eight_in_row8]
eight_outscatter = outscattering(eight_out)
#These are all the calculated matrices
two_migration = np.subtract(np.add(two_absorption,two_outscatter),two_inscatter)
two_B = np.dot(lag.inv(two_migration),two_fission)
#Eigenvalue and vector of the two group
#I keep the eigenvectors in the form numpy gives them, as to not clutter the code with for loops parsing it
#It also makes writing to the output file shorter
two_eigenval,two_eigenvec = lag.eig(two_B)
eight_migration = np.subtract(np.add(eight_absorption,eight_outscatter),eight_inscatter)
eight_B = np.dot(lag.inv(eight_migration),eight_fission)
#Eigenvalue and vector of the eight group
eight_eigenval,eight_eigenvec = lag.eig(eight_B)

#print(two_migration)
#print(two_fission)
print(two_B)
#print(eight_migration)
#print(eight_fission)
#print(two_eigenval)
print(two_eigenvec)
#print(eight_eigenval)
#print(eight_eigenvec)

[[0.15646259 3.87414966]
 [0.03391142 0.83967621]]
[[-0.99918547 -0.97730867]
 [ 0.04035341 -0.21182012]]


In [70]:
#Power iteration
def power_iteration(T,num_sim):
    phi_k = np.ones(len(T[0]))
    k = np.ones(len(T[0]))
    for i in range(num_sim):
        phi_k1 = np.dot(T,phi_k)
        phi_k1_norm = lag.norm(phi_k1)
        phi_k = phi_k1 / phi_k1_norm
        this = np.dot(np.transpose(np.dot(T,phi_k)),phi_k)
        that = np.dot(np.transpose(phi_k),phi_k)
        k = this/that
    return phi_k,k
#The power iterated examples for both problems
two_pow_phi,two_powk = power_iteration(two_B,2)
eight_pow_phi,eight_powk = power_iteration(eight_B,2)
#print(power_iteration(two_B,2))
print(power_iteration(eight_B,2))

(array([0.26071822, 0.34480055, 0.78197216, 0.28035377, 0.22326264,
       0.11761087, 0.10450718, 0.22013545]), 1.0935440381142285)


In [66]:
#Writing to an output file
with open('cp2_output.csv', mode='w') as output_file:
    filewriter = csv.writer(output_file, delimiter=',', lineterminator='\n', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    #This is almost as messy as the output file
    filewriter.writerow(two_sig_a)
    filewriter.writerow(two_fiss_row1)
    filewriter.writerow(two_fiss_row2)
    filewriter.writerow(two_in_row1)
    filewriter.writerow(two_in_row2)
    filewriter.writerow(two_out)
    filewriter.writerow(eight_sig_a)
    filewriter.writerow(eight_fiss_row1)
    filewriter.writerow(eight_fiss_row2)
    filewriter.writerow(eight_fiss_row3)
    filewriter.writerow(eight_in_row1)
    filewriter.writerow(eight_in_row2)
    filewriter.writerow(eight_in_row3)
    filewriter.writerow(eight_in_row4)
    filewriter.writerow(eight_in_row5)
    filewriter.writerow(eight_in_row6)
    filewriter.writerow(eight_in_row7)
    filewriter.writerow(eight_in_row8)
    filewriter.writerow(eight_out)
    filewriter.writerow(two_migration)
    filewriter.writerow(two_fission)
    filewriter.writerow(two_eigenval)
    filewriter.writerow(two_eigenvec)
    filewriter.writerow(two_pow_phi)
    filewriter.writerow([two_powk])
    filewriter.writerow(eight_migration)
    filewriter.writerow(eight_fission)
    filewriter.writerow(eight_eigenval)
    filewriter.writerow(eight_eigenvec)
    filewriter.writerow(eight_pow_phi)
    filewriter.writerow([eight_powk])

In [69]:
#A parser that returns the eigenvectors given the form of the 2d eigenvector array that numpy.linalg.eig gives
#This will only take an 8 by 8 nested list
#This monster returns 8 lists, the flux vectors for each eigenvalue in order
def eigenvec_parser_8(array):
    #like everything else in this, there's probably a better way, but this isn't exactly necessary so i guess its fine
    eig_vec1 = []
    eig_vec2 = []
    eig_vec3 = []
    eig_vec4 = []
    eig_vec5 = []
    eig_vec6 = []
    eig_vec7 = []
    eig_vec8 = []
    for j in range(len(array[0])):
        for i in range(len(array)):
            if j == 0:
                eig_vec1.append(array[i][j])
            elif j == 1:
                eig_vec2.append(array[i][j])
            elif j == 2:
                eig_vec3.append(array[i][j])
            elif j == 3:
                eig_vec4.append(array[i][j])
            elif j == 4:
                eig_vec5.append(array[i][j])
            elif j == 5:
                eig_vec6.append(array[i][j])
            elif j == 6:
                eig_vec7.append(array[i][j])
            elif j == 7:
                eig_vec8.append(array[i][j])
    return eig_vec1,eig_vec2,eig_vec3,eig_vec4,eig_vec5,eig_vec6,eig_vec7,eig_vec8
#print(eight_eigenval)
#print(eigenvec_parser_8(eight_eigenvec))

[ 1.09354393e+00+0.00000000e+00j  2.25573958e-03+0.00000000e+00j
  6.77558387e-06+0.00000000e+00j  2.39370866e-17+0.00000000e+00j
  7.72272390e-18+7.69082255e-18j  7.72272390e-18-7.69082255e-18j
 -3.46567307e-18+1.33951188e-18j -3.46567307e-18-1.33951188e-18j]
([(-0.260717427499532+0j), (-0.34480116824736+0j), (-0.7819721555382609+0j), (-0.2803537689951946+0j), (-0.2232626381790445+0j), (-0.11761086883648067+0j), (-0.10450718366191628+0j), (-0.22013544929027823+0j)], [(-0.762162535794154+0j), (0.6422922059380559+0j), (0.0695540915923728+0j), (0.02599274857269574+0j), (0.020677464488456445+0j), (0.010892934804322288+0j), (0.009679290320449726+0j), (0.02038858406111771+0j)], [(-0.9837784690486108+0j), (0.0015196624933338679+0j), (0.15598796445469265+0j), (0.05528743703996235+0j), (0.04404208594668792+0j), (0.023200358196545534+0j), (0.020615477431319924+0j), (0.043424749261302104+0j)], [(-0.2798092938595023+0j), (-1.3881518619559777e-14+0j), (0.627975774875403+0j), (0.5596185877108041+0j